In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import json
import datetime as dt

from sentinelsat import SentinelAPI
import utils.data_collection as dc
import utils.image_processing as ip
import utils.land_coverage as lc

import ee
ee.Initialize()

In [2]:
geojson_path = "data/geojson_files/istres.geojson"
credentials_path = "secrets/sentinel_api_credentials.json"
wildfire_date = dt.date(2020, 8, 24)
observation_interval = 15
path = 'data/'
output_folder = 'output/'
fire_name = 'istres'

In [ ]:
with open(credentials_path, 'r') as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"],
    credentials["password"]
)

In [ ]:
dc.get_before_after_images(
    api=api,
    wildfire_date=wildfire_date,
    geojson_path=geojson_path,
    observation_interval=observation_interval,
    path=path,
    fire_name=fire_name,
    output_folder=output_folder,
    resolution=10,
    cloud_threshold=40
)

In [ ]:
lat, lon = 43.453228, 4.980225
img_folder = 'data/S2B_MSIL2A_20200906T102559_N0214_R108_T31TFJ_20200906T132601.SAFE/'
pixel_column, pixel_row = ip.get_fire_pixels(img_folder, lat, lon)

In [ ]:
pixel_column, pixel_row

In [ ]:
diff = dc.plot_ndvi_difference(output_folder, fire_name, figsize=(10, 10))
plt.plot(pixel_column, pixel_row, 'ro', markersize=3)
plt.show()

In [ ]:
frag_count = 15
split = ip.split_image(diff, frag_count)

ip.plot_split_image(split, frag_count)
plt.tight_layout()
plt.show()

In [ ]:
fire = ip.merge_images(2, [split[(8, 11)], split[(8, 12)]], horizontal=False)

plt.figure(figsize=(8, 6))
ip.imshow(fire, 'Istres Fire')
plt.show()

In [ ]:
thresholds = np.arange(0.2, 0.41, 0.02)
areas = []
for thr in thresholds:
    tmp = ip.threshold_filter(fire, thr)
    area = round(ip.calculate_area(tmp, diff) * 100, 4)
    areas.append(area)

In [ ]:
true_area = 319.6900

plt.figure(figsize=(8, 6))
plt.plot(thresholds, areas)
plt.hlines(true_area, thresholds[0], thresholds[-1], colors='r', linestyles='dashed')
plt.xlabel('Threshold')
plt.ylabel('Burnt Area (ha)')
plt.title('Istres Fire')
plt.legend(['Calculated', 'True Value'])
plt.show()

In [ ]:
tmp = ip.threshold_filter(fire, 0.28)

plt.figure(figsize=(8, 6))
ip.imshow(tmp, 'Istres Fire')
plt.show()

In [ ]:
print(f'Area: {round(ip.calculate_area(tmp, diff) * 100, 4)} ha')

In [ ]:
v1 = 5750
v2 = 6650
delta_v = v2 - v1
h1 = 8300
h2 = h1 + delta_v

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(diff)
plt.vlines(v1, 0, diff.shape[0], color='r', linestyle='dashed', linewidth=1)
plt.vlines(v2, 0, diff.shape[0], color='r', linestyle='dashed', linewidth=1)
plt.hlines(h1, 0, diff.shape[1], color='r', linestyle='dashed', linewidth=1)
plt.hlines(h2, 0, diff.shape[1], color='r', linestyle='dashed', linewidth=1)
plt.tight_layout()
plt.show()

In [ ]:
fire = diff[h1:h2, v1:v2]
plt.imshow(fire)
plt.axis('off')
plt.savefig('./output/fire_istres.png')
plt.show()

In [ ]:
fire = ip.threshold_filter(fire, 0.28)
plt.imshow(fire)
plt.show()

In [ ]:
rand_image = lc.create_sample_coordinates(fire, 42, 0.1)
ip.imshow(rand_image, 'Sampled Coordinates')
plt.show()

In [ ]:
coordinates = lc.get_coordinates_from_pixels(rand_image, h1, v1, img_folder)
coordinates[:5]

In [ ]:
coordinates = pd.DataFrame(coordinates, columns=['latitude', 'longitude'])
coordinates.to_csv('data/coordinates_files/coords_utm_istres.csv', index=False)
coordinates.head()

In [ ]:
land_cover_data = pd.read_csv('data/MODIS_LandCover_Type1.csv')
land_cover_data

In [ ]:
out_folder = 'output/pie_chart_istres/'
samples = np.arange(50, 1001, 50)
lc.create_plots(
    samples=samples,
    coordinates=coordinates,
    seed=42,
    land_cover_data=land_cover_data,
    fire_name=fire_name,
    out_folder=out_folder,
    save_fig=True
)

In [3]:
lc.make_pie_chart_gif(
    fire_name=fire_name,
    file_path=out_folder,
    save_all=True, duration=500, loop=0
)

<img src='output/pie_chart_istres/istres.gif' width="750" align="center">